# Reading the csv.files in the volume

In [0]:
df = (
    spark.read.option("header", "true").option("inferSchema", "true").csv("/Volumes/workspace/bronze/source_systems/source_crm/cust_info.csv", header=True))
df.display()    

# Write it to Bronz Layer

In [0]:
df.write.mode("overwrite").saveAsTable("bronze.crm_cust_info")

# Check the delta table

In [0]:
%sql
SELECT * FROM workspace.bronze.crm_cust_info LIMIT 10

In [0]:
BASE_PATH = "/Volumes/workspace/bronze/source_systems"

INGESTION_CONFIG = [
   
    {
        "source": "crm",
        "path": f"{BASE_PATH}/source_crm/prd_info.csv",
        "table": "crm_prd_info"
    },
    {
        "source": "crm",
        "path": f"{BASE_PATH}/source_crm/sales_details.csv",
        "table": "crm_sales_details"
    },
    {
        "source": "erp",
        "path": f"{BASE_PATH}/source_erp/CUST_AZ12.csv",
        "table": "erp_cust_az12"
    },
    {
        "source": "erp",
        "path": f"{BASE_PATH}/source_erp/LOC_A101.csv",
        "table": "erp_loc_a101"
    },
    {
        "source": "erp",
        "path": f"{BASE_PATH}/source_erp/PX_CAT_G1V2.csv",
        "table": "erp_px_cat_g1v2"
    }
]

In [0]:
for item in INGESTION_CONFIG:
    print(f"Ingesting {item['source']} → workspace.bronze.{item['table']}")

    df = (
        spark.read
             .option("header", "true")
             .option("inferSchema", "true")
             .csv(item["path"])
    )

    (
        df.write
          .mode("overwrite")
          .format("delta")
          .saveAsTable(f"workspace.bronze.{item['table']}")
    )